<a href="https://colab.research.google.com/github/MartynaPlomecka/DLM_DigitalFootprint/blob/main/updated__main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Apr 26 20:25:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for bert
!pip install transformers
!pip install datasets
import transformers
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
import datasets
from datetime import datetime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 4.0 MB 8.6 MB/s 
     |████████████████████████████████| 6.6 MB 64.4 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
     |████████████████████████████████| 895 kB 65.2 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 8.8 MB/s 
     |████████████████████████████████| 1.1 MB 67.7 MB/s 
     |████████████████████████████████| 136 kB 90.5 MB/s 
     |████████████████████████████████| 212 kB 87.2 MB/s 
     |████████████████████████████████| 127 kB 91.5 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
 

In [4]:
# mount to Google drive
from google.colab import drive
drive.mount('/content/drive')
# dataset paths
dataset_path = '/content/drive/My Drive/embeddings/table4model_clean.csv'
not_labelled_indeed_path = '/content/drive/My Drive/embeddings/indeed_intro.txt'



#not labeled data
dtf = pd.read_csv(not_labelled_indeed_path, header = 0)
dtf

Mounted at /content/drive


,Introduction
0,"Creative problem-solver, who sees the big pict..."
1,To obtain a position as a medical assistant in...
2,Currently seeking employment as an entry level...
3,Around 3+ years of professional IT experience ...
4,Highly motivated sales and marketing represent...
...,...
99995,Work in an environment where my skills and ene...
99996,To work for an organization that allows me to ...
99997,Experienced and highly motivated Shipping & Re...
99998,Over six years of experience in customer servi...


# masked language modelling


Masked Language Model (MLM) is the process how BERT was pre-trained. MLM is a powerful pre-training strategy for learning sentence embeddings. This is especially the case when we work on a specialized domain.



In [7]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
from datetime import datetime
no_deprecation_warning=True

In [8]:
model_name = sys.argv[1]
per_device_train_batch_size = 64

save_steps = 1000               #Save model every 1k steps
num_train_epochs = 3            #Number of epochs
use_fp16 = False                #Set to True, if your GPU supports FP16 operations
max_length = 100                #Max length for a text input
do_whole_word_mask = True       #If set to true, whole words are masked
mlm_prob = 0.15                 #Probability that a word is replaced by a [MASK] token

# Load the model

num_labels=3
#model = BertForMultilabelSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to('cuda')

model = AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased', num_labels=num_labels, problem_type="multi_label_classification")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [9]:
transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM

transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM

In [10]:
 
#output_dir = "output/{}-{}".format(model_name.replace("/", "_"),  datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
output_dir = "output/model"

print("Save checkpoints to:", output_dir)


Save checkpoints to: output/model


In [11]:
##### Load train dataset

train_sentences = []
train_path = '/content/drive/My Drive/embeddings/dev.txt'
with gzip.open(train_path, 'rt', encoding='utf8') if train_path.endswith('.gz') else  open(train_path, 'r', encoding='utf8') as fIn:
    for line in fIn:
        line = line.strip()
        if len(line) >= 10:
            train_sentences.append(line)

print("Train sentences:", len(train_sentences))

dev_sentences = []
if len(sys.argv) >= 4:
    dev_path = sys.argv[3]
    with gzip.open(dev_path, 'rt', encoding='utf8') if dev_path.endswith('.gz') else open(dev_path, 'r', encoding='utf8') as fIn:
        for line in fIn:
            line = line.strip()
            if len(line) >= 10:
                dev_sentences.append(line)

print("Dev sentences:", len(dev_sentences))

#A dataset wrapper, that tokenizes our data on-the-fly
class TokenizedSentencesDataset:
    def __init__(self, sentences, tokenizer, max_length, cache_tokenization=False):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.max_length = max_length
        self.cache_tokenization = cache_tokenization

    def __getitem__(self, item):
        if not self.cache_tokenization:
            return self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)

        if isinstance(self.sentences[item], str):
            self.sentences[item] = self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)
        return self.sentences[item]

    def __len__(self):
        return len(self.sentences)

train_dataset = TokenizedSentencesDataset(train_sentences, tokenizer, max_length)
dev_dataset = TokenizedSentencesDataset(dev_sentences, tokenizer, max_length, cache_tokenization=True) if len(dev_sentences) > 0 else None

##### Training arguments

if do_whole_word_mask:
    data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)
else:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    evaluation_strategy="steps" if dev_dataset is not None else "no",
    per_device_train_batch_size=per_device_train_batch_size,
    eval_steps=save_steps,
    save_steps=save_steps,
    logging_steps=save_steps,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=use_fp16
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

print("Save tokenizer to:", output_dir)
tokenizer.save_pretrained(output_dir)

trainer.train()

print("Save model to:", output_dir)
model.save_pretrained(output_dir)

print("Training done")


Train sentences: 3818
Dev sentences: 0


tokenizer config file saved in output/model/tokenizer_config.json
Special tokens file saved in output/model/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3818
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 180


Save tokenizer to: output/model


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in output/model/config.json


Save model to: output/model


Model weights saved in output/model/pytorch_model.bin


Training done


In [11]:
#data_path = '/content/drive/My Drive/embeddings/dev.txt'

# Model:

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
state_dict = torch.load("output/model/pytorch_model.bin") 
model.load_state_dict(state_dict)

<All keys matched successfully>

## Labeled dataset:

In [13]:
torch.cuda.empty_cache() # PyTorch thing


In [14]:
#labeled:
dtf_l = pd.read_csv(dataset_path, header = 0)
labeled = dtf_l[['About', 'grit', 'bias_to_action', 'resourcefulness']]
labeled

,About,grit,bias_to_action,resourcefulness
0,A passionate Android developer who enjoys buil...,100,75,50
1,Experienced Research Fellow with a demonstrate...,75,25,75
2,"CPQ Product owner, SAP Workstream Lead and Dig...",100,100,75
3,I am a person who has many interests - fashion...,25,75,25
4,"Political scientist, specialized in Audiovisua...",75,25,100
...,...,...,...,...
623,"Aspiring Frontend developer, Homebrewing enthu...",50,100,33
624,For many years I have been interested in widel...,0,0,17
625,"Skills:- SQL, PostgreSQL - basic level,- Java ...",13,17,0
626,"Bilingual Industrial Engineer, with a strong c...",75,67,33


In [16]:
# for debugging:
train = labeled[0:40]
val = labeled[40:55]
test= labeled[55:62]

#splits
#train = labeled[0:400]
#val = labeled[400:550]
#test= labeled[550:]

#to json
labeled_json = labeled.to_json()
labeled.to_json('data.json', lines=True, orient='records')

train_json = train.to_json()
train.to_json('data_train.json', lines=True, orient='records')

val_json = val.to_json()
val.to_json('data_val.json', lines=True, orient='records')

test_json = test.to_json()
test.to_json('data_test.json', lines=True, orient='records')

from datasets import Dataset
dataset = Dataset.from_json('data.json')
train_dataset = Dataset.from_json('data_train.json')
val_dataset = Dataset.from_json('data_val.json')
test_dataset = Dataset.from_json('data_test.json')

Using custom data configuration default-9557bc79ebc120e5


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-8b6d4205266b86dc


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9557bc79ebc120e5/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-d61d501cfa074b25


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-509df86bebc913c2/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d61d501cfa074b25/0.0.0. Subsequent calls will reuse this data.


In [18]:
dataset.train_test_split

<bound method Dataset.train_test_split of Dataset({
    features: ['About', 'grit', 'bias_to_action', 'resourcefulness'],
    num_rows: 628
})>

In [ ]:
#model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)


## Prepare Datasets
### 1.tokenising the dataset by calling tokenizer. 
### 2.associating the label attribute to each dataset item.


In [17]:
#this is done on labeled dataset in jsons

ds = {"train": train_dataset, "validation": val_dataset, "test": test_dataset}

ds = dataset.train_test_split(train_size=500, test_size=100)
def preprocess_function(examples):
    label = examples["grit"] 
    examples = tokenizer(examples["About"], truncation=True, padding="max_length", max_length=256)
    examples["label"] = float(label)
    return examples

cols = ds["train"].column_names
ds = ds.filter(lambda e: all(e[c].isnumeric() for c in cols if c != "About"))
ds = ds.map(lambda x : {"labels": [float(x[c]) for c in cols if c != "About"]})
    

for split in ds:
    ds[split] = ds[split].filter(lambda e: e['grit'].isnumeric())
    ds[split] = ds[split].map(preprocess_function, remove_columns=["About", "grit"])




  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/99 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/99 [00:00<?, ?ex/s]

In [39]:
ds['train'][0]['labels']

[100.0, 25.0, 100.0]

# Metrics
## We can compute metrics to track the model’s improvement during training.
## We retrieve the class with the highest logit (corresponding to the highest probability) for each prediction and compare it with the actual label to calculate the global accuracy score.

We put the output directory for the trained model and the learning parameters into TrainingArguments. With load_best_model_at_end and metric_for_best_model, we will keep several best models (i.e. those with the highest accuracy on the validation set) during training and load the best model at the end.

In [18]:
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../models/",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    print(np.array(labels).shape)
    print(np.array(logits).shape)
    #labels = labels.reshape(-1, 1)
    print(np.array(labels).shape)
    print(np.array(logits).shape)
    logits =[logits[i][0][0:3] for i in range(len(logits))] 
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()

    return {"mse": mse, "mae": mae, "r2": r2}

In [42]:
#tutaj zrob multioutput regression

In [26]:

import torch
torch.cuda.empty_cache()
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        #print(labels.shape)
        outputs = model(**inputs)
        #print(outputs[0][:, 0].size())
        #a = torch.nn.Linear(30522, 1).cuda()
        #logits = a(outputs[0][:, 0].cuda())
        logits = outputs[0][0, 0][0:3]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

        torch.cuda.empty_cache()


In [27]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=compute_metrics_for_regression,
)

torch.cuda.empty_cache() # PyTorch thing


In [23]:
model

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inp

In [28]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: resourcefulness, bias_to_action. If resourcefulness, bias_to_action are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 492
  Num Epochs = 64
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 31488
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Using a target size (torch.Size([1, 3])) that is different to the inp

Epoch,Training Loss,Validation Loss,Mse,Mae,R2
1,No log,689.972778,689.972656,21.024614,-0.036458
2,773.813300,687.818359,687.818298,21.342140,-0.035802
3,770.431900,671.096436,671.096191,21.353889,-0.010011
4,753.307100,697.877869,697.877747,21.787985,-0.051666
5,769.671200,672.965698,672.965393,21.158731,-0.013848
6,758.796900,685.082947,685.083008,21.114687,-0.030345
7,762.392500,672.776733,672.776672,21.199265,-0.013165
8,756.406400,714.768860,714.768738,21.817284,-0.072738
9,756.636400,665.637207,665.637207,21.215515,-0.002151
10,756.090200,667.823853,667.823914,21.338171,-0.005445


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: resourcefulness, bias_to_action. If resourcefulness, bias_to_action are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 99
  Batch size = 1


(99, 3)
(99, 256, 30522)
(99, 3)
(99, 256, 30522)


Saving model checkpoint to ../models/checkpoint-492
Configuration saved in ../models/checkpoint-492/config.json
Model weights saved in ../models/checkpoint-492/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Using a target size (torch.Size([1, 3])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  del sys.path[0]
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: resourcefulness, bias_to_action. If resourcefulness, bias_to_action are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 99
  Batch size = 1


...

(99, 3)
(99, 256, 30522)
(99, 3)
(99, 256, 30522)


Saving model checkpoint to ../models/checkpoint-30996
Configuration saved in ../models/checkpoint-30996/config.json
Model weights saved in ../models/checkpoint-30996/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Using a target size (torch.Size([1, 3])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  del sys.path[0]
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: resourcefulness, bias_to_action. If resourcefulness, bias_to_action are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 99
  Batch size = 1


(99, 3)
(99, 256, 30522)
(99, 3)
(99, 256, 30522)


Saving model checkpoint to ../models/checkpoint-31488
Configuration saved in ../models/checkpoint-31488/config.json
Model weights saved in ../models/checkpoint-31488/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ../models/checkpoint-20172 (score: 664.83837890625).


TrainOutput(global_step=31488, training_loss=750.5729424701474, metrics={'train_runtime': 1673.8163, 'train_samples_per_second': 18.812, 'train_steps_per_second': 18.812, 'total_flos': 2087042841575424.0, 'train_loss': 750.5729424701474, 'epoch': 64.0})

# Evaluation
Note that we rely on the validation set’s accuracy to retrieve the best model. Calling Trainer.evaluate(), we can retrieve the best accuracy attained during training


In [29]:
torch.cuda.empty_cache() 

trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: resourcefulness, bias_to_action. If resourcefulness, bias_to_action are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 99
  Batch size = 1
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Using a target size (torch.Size([1, 3])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  del sys.path[0]


(99, 3)
(99, 256, 30522)
(99, 3)
(99, 256, 30522)


{'epoch': 64.0,
 'eval_loss': 664.83837890625,
 'eval_mae': 21.257522583007812,
 'eval_mse': 664.8383178710938,
 'eval_r2': -0.0009114055520639633,
 'eval_runtime': 8.4858,
 'eval_samples_per_second': 11.667,
 'eval_steps_per_second': 11.667}